# Definition de A1 et A2

In [5]:
class AFD:
    def __init__(self, etats, alphabet, fonction_transition, etat_initial, etats_acceptants):
        self.etats = etats
        self.alphabet = alphabet
        self.fonction_transition = fonction_transition
        self.etat_initial = etat_initial
        self.etats_acceptants = etats_acceptants

    def transition(self, etat, symbole):
        return self.fonction_transition[etat][symbole]

    def accepte(self, chaine):
        etat_courant = self.etat_initial
        for symbole in chaine:
            etat_courant = self.transition(etat_courant, symbole)
        return etat_courant in self.etats_acceptants

    def __repr__(self):
        return (f"AFD(etats={self.etats}, alphabet={self.alphabet}, "
                f"fonction_transition={self.fonction_transition}, "
                f"etat_initial={self.etat_initial}, etats_acceptants={self.etats_acceptants})")

# Exemple d'automate A1
etats_A1 = {'q0', 'q1'}
alphabet_A1 = {'a', 'b'}
fonction_transition_A1 = {
    'q0': {'a': 'q1', 'b': 'q0'},
    'q1': {'a': 'q0', 'b': 'q1'}
}
etat_initial_A1 = 'q0'
etats_acceptants_A1 = {'q1'}

A1 = AFD(etats_A1, alphabet_A1, fonction_transition_A1, etat_initial_A1, etats_acceptants_A1)

# Définition de l'automate A2
etats_A2 = {'p0', 'p1'}
alphabet_A2 = {'a', 'b'}
fonction_transition_A2 = {
    'p0': {'a': 'p0', 'b': 'p1'},
    'p1': {'a': 'p1', 'b': 'p0'}
}
etat_initial_A2 = 'p0'
etats_acceptants_A2 = {'p0'}

A2 = AFD(etats_A2, alphabet_A2, fonction_transition_A2, etat_initial_A2, etats_acceptants_A2)



# Union 

In [9]:
def union(A1, A2):
    etats = A1.etats | A2.etats | {'q_nouveau'}
    alphabet = A1.alphabet | A2.alphabet
    fonction_transition = {}

    # Transition de l'état initial de l'union
    fonction_transition['q_nouveau'] = {char: set() for char in alphabet}
    for char in alphabet:
        if char in A1.alphabet:
            fonction_transition['q_nouveau'][char].add(A1.etat_initial)
        if char in A2.alphabet:
            fonction_transition['q_nouveau'][char].add(A2.etat_initial)

    # Transitions des automates individuels
    for etat in A1.etats:
        fonction_transition[etat] = {char: A1.transition(etat, char) for char in A1.alphabet}
    for etat in A2.etats:
        fonction_transition[etat] = {char: A2.transition(etat, char) for char in A2.alphabet}

    etat_initial = 'q_nouveau'
    etats_acceptants = A1.etats_acceptants | A2.etats_acceptants

    return AFD(etats, alphabet, fonction_transition, etat_initial, etats_acceptants)

A_union = union(A1, A2)
print(A_union)

print(f'\nAutomate A1 :\n\nÉtats : {etats_A1}\nAlphabet : {alphabet_A1}\nFonction de transition :')
for state, transitions in fonction_transition_A1.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {etat_initial_A1}\nÉtats acceptants : {etats_acceptants_A1}')

print(f'\nAutomate A2 :\n\nÉtats : {etats_A2}\nAlphabet : {alphabet_A2}\nFonction de transition :')
for state, transitions in fonction_transition_A2.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {etat_initial_A2}\nÉtats acceptants : {etats_acceptants_A2}')

print(f'\nAutomate Union A1 ∪ A2 :\n\nÉtats : {A_union.etats}\nAlphabet : {A_union.alphabet}\nFonction de transition :')
for state, transitions in A_union.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A_union.etat_initial}\nÉtats acceptants : {A_union.etats_acceptants}')


AFD(etats={'p0', 'p1', 'q0', 'q1', 'q_nouveau'}, alphabet={'b', 'a'}, fonction_transition={'q_nouveau': {'b': {'q0', 'p0'}, 'a': {'q0', 'p0'}}, 'q0': {'b': 'q0', 'a': 'q1'}, 'q1': {'b': 'q1', 'a': 'q0'}, 'p1': {'b': 'p0', 'a': 'p1'}, 'p0': {'b': 'p1', 'a': 'p0'}}, etat_initial=q_nouveau, etats_acceptants={'q1', 'p0'})

Automate A1 :

États : {'q0', 'q1'}
Alphabet : {'b', 'a'}
Fonction de transition :
q0 --a--> q1
q0 --b--> q0
q1 --a--> q0
q1 --b--> q1
État initial : q0
États acceptants : {'q1'}

Automate A2 :

États : {'p1', 'p0'}
Alphabet : {'b', 'a'}
Fonction de transition :
p0 --a--> p0
p0 --b--> p1
p1 --a--> p1
p1 --b--> p0
État initial : p0
États acceptants : {'p0'}

Automate Union A1 ∪ A2 :

États : {'p0', 'p1', 'q0', 'q1', 'q_nouveau'}
Alphabet : {'b', 'a'}
Fonction de transition :
q_nouveau --b--> {'q0', 'p0'}
q_nouveau --a--> {'q0', 'p0'}
q0 --b--> q0
q0 --a--> q1
q1 --b--> q1
q1 --a--> q0
p1 --b--> p0
p1 --a--> p1
p0 --b--> p1
p0 --a--> p0
État initial : q_nouveau
États accep

# Concatenation

In [14]:
def concatener(A1, A2):
    # Union des états et de l'alphabet des deux automates
    etats = A1.etats | A2.etats
    alphabet = A1.alphabet | A2.alphabet
    
    # Initialisation de la fonction de transition
    fonction_transition = {}

    # Copie des transitions de A1
    for etat in A1.etats:
        fonction_transition[etat] = {symbole: A1.transition(etat, symbole) for symbole in A1.alphabet}
    
    # Copie des transitions de A2
    for etat in A2.etats:
        fonction_transition[etat] = {symbole: A2.transition(etat, symbole) for symbole in A2.alphabet}

    # Ajout des transitions pour la concaténation
    for etat_final in A1.etats_acceptants:
        # Pour chaque état final de A1, ajouter une transition vers l'état initial de A2 pour chaque symbole de l'alphabet de A1
        for symbole in A1.alphabet:
            fonction_transition[etat_final][symbole] = A2.etat_initial

    # État initial et états acceptants
    etat_initial = A1.etat_initial
    etats_acceptants = A2.etats_acceptants

    return AFD(etats, alphabet, fonction_transition, etat_initial, etats_acceptants)
# Supposons que A1 et A2 soient des objets AFD déjà créés
A_concat = concatener(A1, A2)
print(A_concat)
# Affichage des détails de l'automate A1
print(f'\nAutomate A1 :\n\nÉtats : {A1.etats}\nAlphabet : {A1.alphabet}\nFonction de transition :')
for state, transitions in A1.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A1.etat_initial}\nÉtats acceptants : {A1.etats_acceptants}')

# Affichage des détails de l'automate A2
print(f'\nAutomate A2 :\n\nÉtats : {A2.etats}\nAlphabet : {A2.alphabet}\nFonction de transition :')
for state, transitions in A2.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A2.etat_initial}\nÉtats acceptants : {A2.etats_acceptants}')

# Affichage des détails de l'automate A1 concaténé avec A2
print(f'\nAutomate Concaténation A1 ∘ A2 :\n\nÉtats : {A_concat.etats}\nAlphabet : {A_concat.alphabet}\nFonction de transition :')
for state, transitions in A_concat.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A_concat.etat_initial}\nÉtats acceptants : {A_concat.etats_acceptants}')


AFD(etats={'p1', 'q0', 'q1', 'p0'}, alphabet={'b', 'a'}, fonction_transition={'q0': {'b': 'q0', 'a': 'q1'}, 'q1': {'b': 'p0', 'a': 'p0'}, 'p1': {'b': 'p0', 'a': 'p1'}, 'p0': {'b': 'p1', 'a': 'p0'}}, etat_initial=q0, etats_acceptants={'p0'})

Automate A1 :

États : {'q0', 'q1'}
Alphabet : {'b', 'a'}
Fonction de transition :
q0 --a--> q1
q0 --b--> q0
q1 --a--> q0
q1 --b--> q1
État initial : q0
États acceptants : {'q1'}

Automate A2 :

États : {'p1', 'p0'}
Alphabet : {'b', 'a'}
Fonction de transition :
p0 --a--> p0
p0 --b--> p1
p1 --a--> p1
p1 --b--> p0
État initial : p0
États acceptants : {'p0'}

Automate Concaténation A1 ∘ A2 :

États : {'p1', 'q0', 'q1', 'p0'}
Alphabet : {'b', 'a'}
Fonction de transition :
q0 --b--> q0
q0 --a--> q1
q1 --b--> p0
q1 --a--> p0
p1 --b--> p0
p1 --a--> p1
p0 --b--> p1
p0 --a--> p0
État initial : q0
États acceptants : {'p0'}


# Etoile de Kleene

In [20]:
def etoile_de_kleene(A):
    # États de l'automate résultant, incluant un nouvel état initial 'q_new'
    etats = A.etats | {'q_new'}
    alphabet = A.alphabet
    # Initialisation de la fonction de transition avec q_new pointant vers l'état initial de A pour chaque symbole de l'alphabet
    fonction_transition = {'q_new': {char: A.etat_initial for char in alphabet}}

    # Copie des transitions de A pour chaque état
    for etat in A.etats:
        fonction_transition[etat] = {char: A.transition(etat, char) for char in alphabet}

    # Ajout des transitions pour la fermeture de Kleene
    for etat_final in A.etats_acceptants:
        # Pour chaque état final de A, ajouter une transition vers l'état initial de A pour chaque symbole de l'alphabet
        for char in alphabet:
            fonction_transition[etat_final][char] = A.etat_initial

    # État initial et états acceptants de l'automate résultant
    etat_initial = 'q_new'
    etats_acceptants = A.etats_acceptants | {'q_new'}

    return AFD(etats, alphabet, fonction_transition, etat_initial, etats_acceptants)

A_etoile = etoile_de_kleene(A1)
print(A_etoile)
# Affichage des détails de l'automate A1
print(f'\nAutomate A1 :\n\nÉtats : {A1.etats}\nAlphabet : {A1.alphabet}\nFonction de transition :')
for state, transitions in A1.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A1.etat_initial}\nÉtats acceptants : {A1.etats_acceptants}')

# Affichage des détails de l'automate résultant de l'étoile de Kleene A1*
print(f'\nAutomate A1* (Étoile de Kleene) :\n\nÉtats : {A_etoile.etats}\nAlphabet : {A_etoile.alphabet}\nFonction de transition :')
for state, transitions in A_etoile.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A_etoile.etat_initial}\nÉtats acceptants : {A_etoile.etats_acceptants}')

AFD(etats={'q_new', 'q0', 'q1'}, alphabet={'b', 'a'}, fonction_transition={'q_new': {'b': 'q0', 'a': 'q0'}, 'q0': {'b': 'q0', 'a': 'q1'}, 'q1': {'b': 'q1', 'a': 'q0'}}, etat_initial=q_new, etats_acceptants={'q_new'})

Automate A1 :

États : {'q0', 'q1'}
Alphabet : {'b', 'a'}
Fonction de transition :
q0 --a--> q1
q0 --b--> q0
q1 --a--> q0
q1 --b--> q1
État initial : q0
États acceptants : set()

Automate A1* (Étoile de Kleene) :

États : {'q_new', 'q0', 'q1'}
Alphabet : {'b', 'a'}
Fonction de transition :
q_new --b--> q0
q_new --a--> q0
q0 --b--> q0
q0 --a--> q1
q1 --b--> q1
q1 --a--> q0
État initial : q_new
États acceptants : {'q_new'}


# Miroir

In [23]:
def miroir(A):
    # États, alphabet et fonction de transition de l'automate miroir
    etats = A.etats
    alphabet = A.alphabet
    # Initialisation de la fonction de transition avec chaque état pointant vers None pour chaque symbole de l'alphabet
    fonction_transition = {etat: {char: None for char in alphabet} for etat in etats}

    # Copie des transitions inversées de A
    for etat in A.etats:
        for char in A.alphabet:
            dest = A.transition(etat, char)
            if dest:
                fonction_transition[dest][char] = etat

    # Déterminer l'état initial de l'automate miroir
    if A.etats_acceptants:
        etat_initial = A.etats_acceptants.pop()
    else:
        # Gérer le cas où il n'y a pas d'état final dans l'automate original
        etat_initial = None  # Vous pouvez choisir de gérer cela selon votre logique métier

    # État acceptant de l'automate miroir (ancien état initial de A)
    etats_acceptants = {A.etat_initial} if A.etat_initial is not None else set()

    return AFD(etats, alphabet, fonction_transition, etat_initial, etats_acceptants)

A_miroir = miroir(A1)
print(A_miroir)

# Affichage des détails de l'automate A1
print(f'\nAutomate A1 :\n\nÉtats : {A1.etats}\nAlphabet : {A1.alphabet}\nFonction de transition :')
for state, transitions in A1.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A1.etat_initial}\nÉtats acceptants : {A1.etats_acceptants}')

# Affichage des détails de l'automate miroir de A1
print(f'\nAutomate Miroir de A1 :\n\nÉtats : {A_miroir.etats}\nAlphabet : {A_miroir.alphabet}\nFonction de transition :')
for state, transitions in A_miroir.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A_miroir.etat_initial}\nÉtats acceptants : {A_miroir.etats_acceptants}')


AFD(etats={'q0', 'q1'}, alphabet={'b', 'a'}, fonction_transition={'q0': {'b': 'q0', 'a': 'q1'}, 'q1': {'b': 'q1', 'a': 'q0'}}, etat_initial=None, etats_acceptants={'q0'})

Automate A1 :

États : {'q0', 'q1'}
Alphabet : {'b', 'a'}
Fonction de transition :
q0 --a--> q1
q0 --b--> q0
q1 --a--> q0
q1 --b--> q1
État initial : q0
États acceptants : set()

Automate Miroir de A1 :

États : {'q0', 'q1'}
Alphabet : {'b', 'a'}
Fonction de transition :
q0 --b--> q0
q0 --a--> q1
q1 --b--> q1
q1 --a--> q0
État initial : None
États acceptants : {'q0'}


# Intersection

In [22]:
def intersection(A1, A2):
    # États de l'automate intersection comme des paires d'états (s1, s2) avec s1 appartenant à A1 et s2 à A2
    etats = {(s1, s2) for s1 in A1.etats for s2 in A2.etats}
    # Alphabet commun aux deux automates
    alphabet = A1.alphabet & A2.alphabet
    # Initialisation de la fonction de transition
    fonction_transition = {}

    # Construction de la fonction de transition pour chaque paire d'états (s1, s2)
    for (s1, s2) in etats:
        fonction_transition[(s1, s2)] = {char: (A1.transition(s1, char), A2.transition(s2, char)) for char in alphabet}

    # État initial de l'automate intersection comme paire d'états initiaux de A1 et A2
    etat_initial = (A1.etat_initial, A2.etat_initial)
    # États acceptants de l'automate intersection comme paires d'états finaux de A1 et A2
    etats_acceptants = {(s1, s2) for s1 in A1.etats_acceptants for s2 in A2.etats_acceptants}

    return AFD(etats, alphabet, fonction_transition, etat_initial, etats_acceptants)

A_intersection = intersection(A1, A2)
print(A_intersection)

# Affichage des détails de l'automate A1
print(f'\nAutomate A1 :\n\nÉtats : {A1.etats}\nAlphabet : {A1.alphabet}\nFonction de transition :')
for state, transitions in A1.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A1.etat_initial}\nÉtats acceptants : {A1.etats_acceptants}')

# Affichage des détails de l'automate A2
print(f'\nAutomate A2 :\n\nÉtats : {A2.etats}\nAlphabet : {A2.alphabet}\nFonction de transition :')
for state, transitions in A2.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A2.etat_initial}\nÉtats acceptants : {A2.etats_acceptants}')

# Affichage des détails de l'automate intersection A1 ∩ A2
print(f'\nAutomate Intersection A1 ∩ A2 :\n\nÉtats : {A_intersection.etats}\nAlphabet : {A_intersection.alphabet}\nFonction de transition :')
for state, transitions in A_intersection.fonction_transition.items():
    for char, next_state in transitions.items():
        print(f'{state} --{char}--> {next_state}')
print(f'État initial : {A_intersection.etat_initial}\nÉtats acceptants : {A_intersection.etats_acceptants}')


AFD(etats={('q0', 'p0'), ('q1', 'p1'), ('q1', 'p0'), ('q0', 'p1')}, alphabet={'b', 'a'}, fonction_transition={('q0', 'p0'): {'b': ('q0', 'p1'), 'a': ('q1', 'p0')}, ('q1', 'p1'): {'b': ('q1', 'p0'), 'a': ('q0', 'p1')}, ('q1', 'p0'): {'b': ('q1', 'p1'), 'a': ('q0', 'p0')}, ('q0', 'p1'): {'b': ('q0', 'p0'), 'a': ('q1', 'p1')}}, etat_initial=('q0', 'p0'), etats_acceptants=set())

Automate A1 :

États : {'q0', 'q1'}
Alphabet : {'b', 'a'}
Fonction de transition :
q0 --a--> q1
q0 --b--> q0
q1 --a--> q0
q1 --b--> q1
État initial : q0
États acceptants : set()

Automate A2 :

États : {'p1', 'p0'}
Alphabet : {'b', 'a'}
Fonction de transition :
p0 --a--> p0
p0 --b--> p1
p1 --a--> p1
p1 --b--> p0
État initial : p0
États acceptants : {'p0'}

Automate Intersection A1 ∩ A2 :

États : {('q0', 'p0'), ('q1', 'p1'), ('q1', 'p0'), ('q0', 'p1')}
Alphabet : {'b', 'a'}
Fonction de transition :
('q0', 'p0') --b--> ('q0', 'p1')
('q0', 'p0') --a--> ('q1', 'p0')
('q1', 'p1') --b--> ('q1', 'p0')
('q1', 'p1') --a--

# Automates à Pile

In [29]:
class AP:
    def __init__(self):
        self.pile = []  # Initialisation de la pile vide
        self.etat = 'q0'  # Initialisation de l'état à 'q0'

    def transition(self, symbole_entree):
        # Si l'automate est dans l'état 'q0'
        if self.etat == 'q0':
            if symbole_entree == 'a':
                self.pile.append('a')  # Empile 'a'
                self.etat = 'q0'  # Reste dans l'état 'q0'
            elif symbole_entree == 'b' and self.pile:
                self.pile.pop()  # Dépile si la pile n'est pas vide
                self.etat = 'q1'  # Passe à l'état 'q1'
        # Si l'automate est dans l'état 'q1'
        elif self.etat == 'q1':
            if symbole_entree == 'b' and self.pile:
                self.pile.pop()  # Dépile si la pile n'est pas vide
                self.etat = 'q1'  # Reste dans l'état 'q1'

    def est_acceptee(self):
        # La chaîne est acceptée si l'état est 'q1' et la pile est vide
        return self.etat == 'q1' and not self.pile

    def traiter_entree(self, chaine_entree):
        for symbole in chaine_entree:
            self.transition(symbole)
        return self.est_acceptee()

# Tester l'automate
ap = AP()
chaine_entree = "bb"  # Changer la chaîne d'entrée pour tester différentes entrées
if ap.traiter_entree(chaine_entree):
    print(f"La chaîne '{chaine_entree}' est acceptée.")
else:
    print(f"La chaîne '{chaine_entree}' est rejetée.")


La chaîne 'bb' est rejetée.


In [30]:
class TuringMachine:
    def __init__(self, tape="", blank_symbol=" ", initial_state="q0", final_states=None):
        self.tape = list(tape)  # Convertir la bande en une liste de caractères
        self.blank_symbol = blank_symbol  # Symbole représentant une case vide sur la bande
        self.head_position = 0  # Position de la tête de lecture/écriture
        self.current_state = initial_state  # État initial de la machine
        self.final_states = final_states if final_states is not None else set()  # Ensemble des états finaux
        self.transition_function = {}  # Dictionnaire pour stocker la fonction de transition
        
    def add_transition(self, state, symbol, new_state, new_symbol, direction):
        # Ajouter une transition à la fonction de transition
        self.transition_function[(state, symbol)] = (new_state, new_symbol, direction)
    
    def step(self):
        # Exécuter une transition basée sur l'état actuel et le symbole sous la tête de lecture
        current_symbol = self.tape[self.head_position] if self.head_position < len(self.tape) else self.blank_symbol
        if (self.current_state, current_symbol) in self.transition_function:
            new_state, new_symbol, direction = self.transition_function[(self.current_state, current_symbol)]
            if self.head_position < len(self.tape):
                self.tape[self.head_position] = new_symbol
            else:
                self.tape.append(new_symbol)
            self.current_state = new_state
            if direction == 'R':
                self.head_position += 1
            elif direction == 'L':
                self.head_position -= 1
            if self.head_position < 0:
                self.tape.insert(0, self.blank_symbol)
                self.head_position = 0
        else:
            raise Exception(f"No transition defined for ({self.current_state}, {current_symbol})")
    
    def run(self):
        # Exécuter la machine jusqu'à ce qu'elle atteigne un état final
        while self.current_state not in self.final_states:
            self.step()
    
    def get_tape_content(self):
        # Obtenir le contenu de la bande sans les symboles vides initiaux et finaux
        return "".join(self.tape).strip(self.blank_symbol)
    
    def __str__(self):
        # Représenter l'état actuel de la machine, utile pour le débogage
        tape_str = "".join(self.tape).strip(self.blank_symbol)
        head_indicator = " " * self.head_position + "^"
        return f"Tape: {tape_str}\nHead: {head_indicator}\nState: {self.current_state}"

# Définir la machine de Turing
tm = TuringMachine(tape="aaabbbccc", final_states={"q_accept"})

# Ajouter les transitions
# Exemple simplifié pour a^n b^n c^n
transitions = {
    ("q0", "a"): ("q0", "a", "R"),
    ("q0", "b"): ("q1", "X", "R"),
    ("q1", "b"): ("q1", "b", "R"),
    ("q1", "c"): ("q2", "Y", "R"),
    ("q2", "c"): ("q2", "c", "R"),
    ("q2", " "): ("q_accept", " ", "R"),
}

for (state, symbol), (new_state, new_symbol, direction) in transitions.items():
    tm.add_transition(state, symbol, new_state, new_symbol, direction)

# Exécuter la machine de Turing
print("Contenu initial de la bande:", tm.get_tape_content())
tm.run()
print("Contenu final de la bande:", tm.get_tape_content())
print("État final:", tm.current_state)


Contenu initial de la bande: aaabbbccc
Contenu final de la bande: aaaXbbYcc
État final: q_accept
